In [149]:
import pandas as pd
import datetime
import os
directory = 'data\\as_csv'

In [150]:
# Create empty data frame to populate with activities
df_activities_cloud = pd.DataFrame([], columns=['latitude','longitude','altitude', 'r', 'g', 'b'])

#combine all activities into 1 dataset. Drop activities before July 2021--the month I moved to Seattle 
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    df = pd.read_csv(f)
    date_string = df['time'][0][:10]
    date_list = date_string.split('-')
    datetime_obj = datetime.datetime(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    if datetime_obj < datetime.datetime(2021,7,1):
        continue
        
    # Drop time column, and every other row. Data points are dense enough to display adequately with half as many.
    df = df.drop(['time'], axis=1)
    df = df.iloc[::2]
    
    # increase scale of altitude to increase 3D appearance
    df['altitude'] = df['altitude']*2.5
    
    # add rgb values based on normalized location data
    df_normalized = (df-df.min())/(df.max()-df.min())
    df.insert(3, 'r', (df_normalized['latitude']*255).astype(int))
    df.insert(4, 'g', (df_normalized['longitude']*255).astype(int))
    df.insert(5, 'b', (df_normalized['altitude']*255).astype(int))

    # Add activity coordinates to the activity
    df_activities_cloud = pd.concat([df_activities_cloud, df])    

In [151]:
df_activities_cloud.to_csv('data/activities_cloud.csv', index=False)
